In [ ]:
import pandas as pd
import re
import pandas_datareader as dr

from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
%store -r

In [ ]:
# %run PrepareData.ipynb
%run Stock_direction_predictor_RF.ipynb

In [ ]:
tsla_df = df.copy()
df.plot(y="close_value")
# display(tsla_df.head())
# display(tsla_df.tail())

In [ ]:
# def get_training_testing_data(dataframe,random=False,test_size=.25):
#     if(random):
#         # Split the dataset using train_test_split
#         X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=test_size,random_state=1)
#     else:
        
            

In [ ]:
train_data_size = int (len(tsla_df) * .7)
test_data_size = len(tsla_df) - train_data_size

y = tsla_df["price_direction"]
X = tsla_df.drop(["price_direction"],axis=1)

In [ ]:
train_start = X.index.min()
train_end = X.index.min() + DateOffset(days=train_data_size)
X_train = X.loc[train_start:train_end]
y_train = y.loc[train_start:train_end]
display(X_train.info())

In [ ]:
test_start = train_end + DateOffset(days=1)
X_test = X.loc[test_start:]
y_test = y.loc[test_start:]
display(X_test.info())

In [ ]:
# Split the dataset using train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
scaler = StandardScaler()

# Fitting Standard Scaler
X_scaler = scaler.fit(X_train)

# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

display(X_test_scaled)

In [ ]:
# find optimal value of neighbors, by evaluating accuracy score by cross validation method
k_neighbors = [num for num in range(1,50,2)]

k_acc_scores = []

for k in k_neighbors:
    knn = KNeighborsClassifier(n_neighbors=k)
    cv_scores = cross_val_score(knn, X_train_scaled,y_train,cv=5, scoring="accuracy")
    k_acc_scores.append(cv_scores.mean())

optimal_k_neighbors = k_neighbors[k_acc_scores.index(max(k_acc_scores))]

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors=optimal_k_neighbors)
# Train model using training data
knn_model.fit(X_train_scaled, y_train)

In [ ]:
# Create predictions using the testing data
y_pred = knn_model.predict(X_test_scaled)
print(y_pred)
print(y_test)

In [ ]:
# Print the classification report comparing the testing data to the model predictions
print(classification_report(y_test, y_pred))